### Install Qiskit

In [2]:
pip install qiskit

     |████████████████████████████████| 1.5MB 5.1MB/s 
     |████████████████████████████████| 21.3MB 1.0MB/s 
     |████████████████████████████████| 61kB 22.3MB/s 
     |████████████████████████████████| 81kB 27.2MB/s 
     |████████████████████████████████| 409kB 44.9MB/s 
     |████████████████████████████████| 522kB 40.5MB/s 
     |████████████████████████████████| 51kB 19.1MB/s 
     |████████████████████████████████| 5.3MB 32.7MB/s 
     |████████████████████████████████| 51kB 19.1MB/s 
     |████████████████████████████████| 51kB 21.2MB/s 
     |████████████████████████████████| 71kB 24.3MB/s 
     |████████████████████████████████| 532kB 46.4MB/s 
     |████████████████████████████████| 21.5MB 22.6MB/s 
     |████████████████████████████████| 2.3MB 32.2MB/s 
     |████████████████████████████████| 61kB 22.9MB/s 
     |████████████████████████████████| 61kB 23.3MB/s 
     |████████████████████████████████| 102kB 29.0MB/s 
  ERROR: Failed building wheel for qiskit
  Running setu

In [0]:
# import common packages
import numpy as np

from qiskit import Aer

# lib from Qiskit Aqua
from qiskit.aqua import Operator, QuantumInstance
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import COBYLA

# lib from Qiskit Aqua Chemistry
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock

### Set up the molecular structure

In [0]:
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.72', unit=UnitsType.ANGSTROM, basis='sto3g')
molecule = driver.run()

In [92]:
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals
nuclear_repulsion_energy = molecule.nuclear_repulsion_energy

num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2
print("HF energy: {}".format(molecule.hf_energy))
print("# of electrons: {}".format(num_particles))
print("# of spin orbitals: {}".format(num_spin_orbitals))

HF energy: -1.1174450773714701
# of electrons: 2
# of spin orbitals: 4


### Choose your mapping

In [0]:
map_type = 'jordan_wigner'

### Generate the Pauli strings

In [24]:
ferOp = FermionicOperator(h1=h1, h2=h2)
qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)


print(qubitOp.print_operators())
print(qubitOp)

IIII	(-0.8362123048044218+0j)
IIIZ	(0.08705553358137133+0j)
IIZI	(-0.024253213726949913+0j)
IZII	(0.08705553358137129+0j)
ZIII	(-0.02425321372694993+0j)
IIZZ	(0.07854309838088275+0j)
IZIZ	(0.13546887571547914+0j)
XXYY	(0.05897532134962703+0j)
YYYY	(0.05897532134962703+0j)
XXXX	(0.05897532134962703+0j)
YYXX	(0.05897532134962703+0j)
ZIIZ	(0.1375184197305098+0j)
IZZI	(0.1375184197305098+0j)
ZIZI	(0.14301575315731474+0j)
ZZII	(0.07854309838088275+0j)

Representation: paulis, qubits: 4, size: 15


### Set up the Hartree Fock states

In [0]:
qubit_reduction = False
HF_state = HartreeFock(qubitOp.num_qubits, num_spin_orbitals, num_particles, map_type, 
                       qubit_reduction)

### Set up the UCCSD ansatzs

In [58]:
uccsd_ansatz = UCCSD(qubitOp.num_qubits, depth=1, 
                   num_orbitals=num_spin_orbitals, num_particles=num_particles,
                   initial_state=HF_state, qubit_mapping=map_type, 
                   two_qubit_reduction=qubit_reduction)
list = np.random.rand(3)
print(list)
uccsd_ansatz_circuit = uccsd_ansatz.construct_circuit(list)

[0.8492349  0.28429897 0.57207906]


### Visualize the UCCSD ansatz circuit

In [59]:
print(uccsd_ansatz_circuit)

        ┌─────────────────────┐┌──────────┐                         »
q_0: |0>┤ U3(3.1416,0,3.1416) ├┤ U2(0,pi) ├──■───────────────────■──»
        ├─────────────────────┤└──────────┘┌─┴─┐┌─────────────┐┌─┴─┐»
q_1: |0>┤ U3(pi/2,-pi/2,pi/2) ├────────────┤ X ├┤ U1(0.84923) ├┤ X ├»
        ├─────────────────────┤┌──────────┐└───┘└─────────────┘└───┘»
q_2: |0>┤ U3(3.1416,0,3.1416) ├┤ U2(0,pi) ├──■───────────────────■──»
        ├─────────────────────┤└──────────┘┌─┴─┐ ┌────────────┐┌─┴─┐»
q_3: |0>┤ U3(pi/2,-pi/2,pi/2) ├────────────┤ X ├─┤ U1(0.2843) ├┤ X ├»
        └─────────────────────┘            └───┘ └────────────┘└───┘»
«           ┌──────────┐      ┌─────────────────────┐                          »
«q_0: ──────┤ U2(0,pi) ├──────┤ U3(pi/2,-pi/2,pi/2) ├──■────────────────────■──»
«     ┌─────┴──────────┴─────┐└─────┬──────────┬────┘┌─┴─┐┌──────────────┐┌─┴─┐»
«q_1: ┤ U3(-pi/2,-pi/2,pi/2) ├──────┤ U2(0,pi) ├─────┤ X ├┤ U1(-0.84923) ├┤ X ├»
«     └─────┬──────────┬─────┘┌─────┴─────────

### Convert the circuit to OpenQASM code

In [60]:
print(uccsd_ansatz_circuit.qasm())

OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
u3(3.14159265358979,0.0,3.14159265358979) q[0];
u3(3.14159265358979,0.0,3.14159265358979) q[2];
u2(0.0,pi) q[0];
u3(pi/2,-pi/2,pi/2) q[1];
cx q[0],q[1];
u1(0.849234900739678) q[1];
cx q[0],q[1];
u2(0.0,pi) q[0];
u3(-pi/2,-pi/2,pi/2) q[1];
u3(pi/2,-pi/2,pi/2) q[0];
u2(0.0,pi) q[1];
cx q[0],q[1];
u1(-0.849234900739678) q[1];
cx q[0],q[1];
u3(-pi/2,-pi/2,pi/2) q[0];
u2(0.0,pi) q[1];
u2(0.0,pi) q[2];
u3(pi/2,-pi/2,pi/2) q[3];
cx q[2],q[3];
u1(0.284298969229392) q[3];
cx q[2],q[3];
u2(0.0,pi) q[2];
u3(-pi/2,-pi/2,pi/2) q[3];
u3(pi/2,-pi/2,pi/2) q[2];
u2(0.0,pi) q[3];
cx q[2],q[3];
u1(-0.284298969229392) q[3];
cx q[2],q[3];
u3(-pi/2,-pi/2,pi/2) q[2];
u2(0.0,pi) q[3];
u2(0.0,pi) q[0];
u2(0.0,pi) q[1];
u2(0.0,pi) q[2];
u3(pi/2,-pi/2,pi/2) q[3];
cx q[0],q[1];
cx q[1],q[2];
cx q[2],q[3];
u1(0.143019766019107) q[3];
cx q[2],q[3];
cx q[1],q[2];
cx q[0],q[1];
u2(0.0,pi) q[0];
u2(0.0,pi) q[1];
u2(0.0,pi) q[2];
u3(-pi/2,-pi/2,pi/2) q[3];
u2(0.0,pi) q[0];